In [1]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embedding_model_name = '../models/bge-m3'
# embedding_model_name = 'kornwtp/simcse-model-phayathaibert'
embed_model = HuggingFaceEmbedding(model_name=embedding_model_name,max_length=1024, device='cpu')

In [2]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

In [3]:
from llama_index.core import Settings

Settings.llm = None
Settings.embed_model = embed_model

LLM is explicitly disabled. Using MockLLM.


In [4]:
from llama_index.core import (

    load_index_from_storage,
    StorageContext,
)

from IPython.display import Markdown, display
from llama_index.vector_stores.faiss import FaissVectorStore


In [5]:
persist_dir="../index"
vector_store = FaissVectorStore.from_persist_dir(persist_dir)
storage_context = StorageContext.from_defaults(vector_store=vector_store, persist_dir=persist_dir)
index = load_index_from_storage(storage_context=storage_context)

In [6]:
query_enging = index.as_query_engine()


In [7]:
question = 'แนะนำที่เที่ยวจังหวัดอยุธยาให้หน่อย'
response_ = query_enging.query(question)

In [8]:
print(response_.response)

Context information is below.
---------------------
==

ศูนย์ท่องเที่ยวอยุธยา (ATC)
ศูนย์ศึกษาประวัติศาสตร์อยุธยา
พิพิธภัณฑสถานแห่งชาติ เจ้าสามพระยา
อุทยานประวัติศาสตร์พระนครศรีอยุธยา
วัดพระศรีสรรเพชญ์
พระราชวังโบราณ อยุธยา
พระที่นั่งวิหารสมเด็จ
พระที่นั่งสรรเพชญปราสาท
พระที่นั่งสุริยาสน์อมรินทร์

เมื่อวันที่ 13 ธันวาคม พ.ศ. 2534 มีพื้นที่ครอบคลุมในบริเวณโบราณสถานเมืองอยุธยา


== ภูมิศาสตร์ ==


=== สภาพภูมิอากาศ ===


== สัญลักษณ์ประจำจังหวัด ==
คำขวัญประจำจังหวัด : ราชธานีเก่า อู่ข้าวอู่น้ำ เลิศล้ำกานท์กวี คนดีศรีอยุธยา เลอคุณค่ามรดกโลก
ตราประจำจังหวัด :
---------------------
Given the context information and not prior knowledge, answer the query.
Query: แนะนำที่เที่ยวจังหวัดอยุธยาให้หน่อย
Answer: 


In [9]:
import json

In [10]:
import requests
def send_api2(prompt,
             url ='http://127.0.0.1:8080/completion',
             temp = 0.7,
             n_predict= 512,
            ):
    payload = json.dumps({
    "prompt": f'''<s>[INST] <<SYS>>
    You are a question answering assistant. Answer the question as truthful and helpful as possible
    คุณคือผู้ช่วยตอบคำถาม จงตอบคำถามอย่างถูกต้องและมีประโยชน์ที่สุด
    <</SYS>>

    Answer the question based only on the following context:
    {prompt}

    [/INST]''',
    "n_predict": n_predict,
    'temperature': temp,
    "stop": ["</s>"]
    })

    headers = {
    'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    
    return response.json()

In [11]:
def generate_answer(question):
    response_ = query_enging.query(question)
    res_ = send_api2(prompt=response_.response)
    return res_

In [12]:
generate_answer('อยุธยามีกี่อำเภอ')

{'content': 'จังหวัดพระนครศรีอยุธยามีทั้งหมด 16 อำเภอ ได้แก่ อำเภอพระนครศรีอยุธยา, อำเภอท่าเรือ, อำเภอนครหลวง, อำเภอบางบาล, อำเภอบางปะหัน, อำเภอภาชี, อำเภอบางปะหัน, อำเภอบ้านแพรก, อำเภอบ้านหมอ, อำเภอบางขุนเทียน, อำเภอบ้านเมาะ, อำเภออุทัย, อำเภอบางปะอิน, อำเภอบางขุนเทียน, อำเภอบ้านโป่ง, อำเภอโพน อำเภอบางระจัน และอำเภอลาดบัวหลวง',
 'id_slot': 0,
 'stop': True,
 'model': '/home/search/code/models/openthaigpt-Q4_K_M.gguf',
 'tokens_predicted': 128,
 'tokens_evaluated': 281,
 'generation_settings': {'n_ctx': 512,
  'n_predict': -1,
  'model': '/home/search/code/models/openthaigpt-Q4_K_M.gguf',
  'seed': 4294967295,
  'temperature': 0.699999988079071,
  'dynatemp_range': 0.0,
  'dynatemp_exponent': 1.0,
  'top_k': 40,
  'top_p': 0.949999988079071,
  'min_p': 0.05000000074505806,
  'tfs_z': 1.0,
  'typical_p': 1.0,
  'repeat_last_n': 64,
  'repeat_penalty': 1.0,
  'presence_penalty': 0.0,
  'frequency_penalty': 0.0,
  'penalty_prompt_tokens': [],
  'use_penalty_prompt_tokens': False,
  'miros